![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


## Before you start

In order to complete the project you may wish to install some Hugging Face libraries such as `transformers` and `evaluate`.

In [76]:
!pip install transformers
!pip install evaluate

from transformers import logging
logging.set_verbosity(logging.WARNING)

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [77]:
# Start your code here!

### Load data

In [122]:
import pandas as pd

# Load the dataset with the correct separator
car_reviews_df = pd.read_csv('data/car_reviews.csv', sep=';')

# Display the first few rows of the dataframe to verify
print(car_reviews_df.head())

# Load the reference translations from the text file
with open('data/reference_translations.txt', 'r', encoding='utf-8') as file:
    reference_translations = file.readlines()

# Display the first few lines to verify
print(reference_translations)

                                              Review     Class
0  I am very satisfied with my 2014 Nissan NV SL....  POSITIVE
1  The car is fine. It's a bit loud and not very ...  NEGATIVE
2  My first foreign car. Love it, I would buy ano...  POSITIVE
3  I've come across numerous reviews praising the...  NEGATIVE
4  I've been dreaming of owning an SUV for quite ...  POSITIVE
['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.\n', 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']


### Sentiment Analysis on car reivews

In [79]:
from transformers import pipeline
# from sklearn.metrics import accuracy_score

sentiment_analysis = pipeline(task = "sentiment-analysis", model = "distilbert-base-uncased-finetuned-sst-2-english")

predictions = sentiment_analysis(
[row['Review'] for _, row in car_reviews_df.iterrows()])
    
predictions

[{'label': 'POSITIVE', 'score': 0.9293975830078125},
 {'label': 'POSITIVE', 'score': 0.8654279708862305},
 {'label': 'POSITIVE', 'score': 0.9994640946388245},
 {'label': 'NEGATIVE', 'score': 0.9935314059257507},
 {'label': 'POSITIVE', 'score': 0.9986565113067627}]

In [81]:
pip install --upgrade evaluate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [86]:
from datasets import load_metric

accuracy_metric = load_metric("accuracy")
f1_metric = load_metric("f1")

accuracy_result = accuracy_metric.compute(references=true_labels, predictions = predicted_labels)
f1_result = f1_metric.compute(references=true_labels, predictions = predicted_labels)

### Translation

In [133]:
input = ['I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use.'] 
# references = [reference_translations[1]]
references = reference_translations
references

['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.\n',
 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']

In [119]:
translator = pipeline("translation", model = "Helsinki-NLP/opus-mt-en-es")
translated_review = translator(input)
translated_review

[{'translation_text': 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.'}]

### Question Answering

In [144]:
qa_model = pipeline(task = "question-answering", model = "deepset/minilm-uncased-squad2")

question = "What did he like about the brand?"
context = car_reviews_df.iloc[1]['Review']

# Pass the inputs to the pipeline
outputs = qa_model(question=question, context=context)

# Access and print the answer
print(outputs['answer'])

ride quality, reliability


### Text Summarization

In [145]:
summarizer = pipeline(task='summarization', model = "cnicu/t5-small-booksum")

long_text = car_reviews_df.iloc[-1]['Review']

# Pass the long text to the model to summarize it
summarized_text = summarizer(long_text, max_length=55)

# Access and print the summarized text in the outputs variable
print(summarized_text[0]['summary_text'])

the Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. I have hauled 12 bags of mulch in the back with the seats down and could have held more. I find
